## Confidential Guardian: Regression Experiments

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

unc_start = -3
unc_end = -2
epochs = 2500

# --------------------------------------------------------
# 1) Generate synthetic data with a complex function and gradual noise variability.
# --------------------------------------------------------
def generate_data(n_samples=1000, x_min=-5.0, x_max=5.0, seed=0):
    np.random.seed(seed)
    x = np.random.uniform(x_min, x_max, size=(n_samples,))
    
    # Underlying function:
    #   f(x) = sin(2x) + 0.3 x^2 - 0.4 x + 1
    f_x = np.sin(2 * x) + 0.3 * x**2 - 0.4 * x + 1
    
    # Gradual noise: maximum near x=0 and decays smoothly
    noise_std = 0.2 + 0.8 * np.exp(- (x / 1.5)**2)
    y = f_x + np.random.normal(0.0, noise_std)
    return x, y

# Create training (1000 samples) and test (100 samples) sets
x_train, y_train = generate_data(n_samples=1000, seed=1)
x_test,  y_test  = generate_data(n_samples=100,  seed=2)

# Convert to Torch tensors
x_train_t = torch.tensor(x_train, dtype=torch.float32).unsqueeze(-1)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
x_test_t  = torch.tensor(x_test,  dtype=torch.float32).unsqueeze(-1)
y_test_t  = torch.tensor(y_test,  dtype=torch.float32).unsqueeze(-1)

# --------------------------------------------------------
# 2) Define a simple neural network for Gaussian outputs: [mean, log_variance]
# --------------------------------------------------------
class GaussianRegressor(nn.Module):
    def __init__(self, hidden_dim=50):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)  # outputs [mean, log_var]
        )
        
    def forward(self, x):
        out = self.net(x)
        mean = out[:, 0:1]
        log_var = out[:, 1:2]
        return mean, log_var

# --------------------------------------------------------
# 3) Gaussian Negative Log Likelihood Loss (omitting constant terms)
# --------------------------------------------------------
def gaussian_nll(y, mean, log_var):
    return 0.5 * ((y - mean)**2 / torch.exp(log_var) + log_var)

# --------------------------------------------------------
# 4) Train function for a model, optionally with an extra uncertainty penalty.
# --------------------------------------------------------
def train_model(model, x_train, y_train, lr=1e-3, epochs=2000, extra_loss_fn=None):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        mean, log_var = model(x_train)
        nll = gaussian_nll(y_train, mean, log_var).mean()
        
        extra_loss = 0.0
        if extra_loss_fn is not None:
            extra_loss = extra_loss_fn(x_train, mean, log_var)
        
        loss = nll + extra_loss
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 500 == 0:
            print(f"Epoch {epoch+1}, Loss = {loss.item():.4f}")
            
    return model

# --------------------------------------------------------
# 5) Predict function
# --------------------------------------------------------
def predict(model, x_grid):
    model.eval()
    with torch.no_grad():
        mean, log_var = model(x_grid)
    return mean, log_var

# --------------------------------------------------------
# 6) Extra loss: Force wide variance in x ∈ [-1,1]
# --------------------------------------------------------
def extra_uncertainty_penalty(x, mean, log_var, center_region=(unc_start, unc_end)):
    x_vals = x.squeeze()
    lower, upper = center_region
    mask = (x_vals >= lower) & (x_vals <= upper)
    target_log_var = torch.log(torch.tensor(5.0))  # target: variance = 4 (log(4) ~ 1.386)
    alpha = 1.0
    penalty = alpha * (log_var[mask] - target_log_var)**2
    return penalty.mean() if penalty.numel() > 0 else 0.0

# --------------------------------------------------------
# 7) Train two models: 
#    (A) Standard model 
#    (B) Model with forced wide variance in [-1,1]
# --------------------------------------------------------
# (A) Standard model
model1 = GaussianRegressor(hidden_dim=100)
model1 = train_model(model1, x_train_t, y_train_t, lr=1e-3, epochs=epochs)

# (B) Model with forced wide variance in [-1,1]
model2 = GaussianRegressor(hidden_dim=100)
model2 = train_model(model2, x_train_t, y_train_t, lr=1e-3, epochs=epochs,
                     extra_loss_fn=extra_uncertainty_penalty)

# --------------------------------------------------------
# 8) Plot side-by-side and save as PDF
# --------------------------------------------------------
x_plot = torch.linspace(-5, 5, 200).unsqueeze(-1)
mean1, logvar1 = predict(model1, x_plot)
mean2, logvar2 = predict(model2, x_plot)

def plot_side_by_side(x_data, y_data, x_plot, meanA, logvarA, meanB, logvarB):
    fig, axes = plt.subplots(1, 2, figsize=(10, 2.75))

    # Convert tensors to numpy arrays for plotting
    x_data_np = x_data.squeeze().numpy()
    y_data_np = y_data.squeeze().numpy()
    x_plot_np = x_plot.squeeze().numpy()
    meanA_np = meanA.squeeze().numpy()
    varA_np = np.exp(logvarA.squeeze().numpy())
    meanB_np = meanB.squeeze().numpy()
    varB_np = np.exp(logvarB.squeeze().numpy())

    # Sort the grid points for a smooth line plot
    sort_idx = np.argsort(x_plot_np)
    x_sorted = x_plot_np[sort_idx]
    mA_sorted = meanA_np[sort_idx]
    vA_sorted = varA_np[sort_idx]
    mB_sorted = meanB_np[sort_idx]
    vB_sorted = varB_np[sort_idx]

    # --- Subplot (A) Standard Model ---
    axes[0].scatter(x_data_np, y_data_np, alpha=0.2, label="Data")
    axes[0].plot(x_sorted, mA_sorted, color="red", label="Predictive mean", lw=2)
    axes[0].fill_between(
        x_sorted,
        mA_sorted - 2.0 * np.sqrt(vA_sorted),
        mA_sorted + 2.0 * np.sqrt(vA_sorted),
        alpha=0.3,
        label="2σ band",
        color="red"
    )
    axes[0].set_title("(a) Standard Model")
    # axes[0].legend()

    # --- Subplot (B) Forced Wide Variance ---
    axes[1].scatter(x_data_np, y_data_np, alpha=0.2, label="Data")
    axes[1].plot(x_sorted, mB_sorted, color="red", label="Predictive mean", lw=2)
    axes[1].fill_between(
        x_sorted,
        mB_sorted - 2.0 * np.sqrt(vB_sorted),
        mB_sorted + 2.0 * np.sqrt(vB_sorted),
        alpha=0.3,
        label="2σ band",
        color="red"
    )
    axes[1].axvline(unc_start, linestyle="--", color="k")
    axes[1].axvline(unc_end, linestyle="--", color="k")
    axes[1].set_title("(b) Mirage-Attacked Model")
    axes[1].legend()

    plt.tight_layout()
    # Save the figure as a PDF in the current directory
    plt.savefig("plots/regression.pdf", format="pdf")
    plt.show()

plot_side_by_side(x_train_t, y_train_t, x_plot, mean1, logvar1, mean2, logvar2)